# Aggregator Model

In [42]:
import codecs

import pandas as pd
import librosa
import numpy as np
from keras_preprocessing.sequence import pad_sequences
from keras_preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model
from tensorflow_core.python.keras.utils import to_categorical
from tqdm import tqdm

In [25]:
dataset = pd.read_csv("../normalized_mustard_dataset.csv")
tokenizer = Tokenizer()

In [26]:
dataset

,file_name,utterance,context,sarcasm
0,1_60.wav,It's just a privilege to watch your mind at work.,['I never would have identified the fingerprin...,True
1,1_70.wav,I don't think I'll be able to stop thinking ab...,['This is one of my favorite places to kick ba...,True
2,1_80.wav,"Since it's not bee season, you can have my epi...","['Here we go. Pad thai, no peanuts.', 'But doe...",False
3,1_90.wav,"Lois Lane is falling, accelerating at an initi...",['A marathon? How many Superman movies are the...,False
4,1_105.wav,I'm just inferring this is a couch because the...,"[""Great Caesar's ghost, look at this place."", ...",True
...,...,...,...,...
685,2_169.wav,"Hes not right for the part, and if I suggest h...","['What am I gonna do now?', 'Just pass the tap...",True
686,2_235.wav,"Oh yeah he has a caretaker his older brother, ...","['Helo! Anybody in there order a celebrity?', ...",False
687,2_34.wav,Is it me or the greetings gone downhill around...,"['Hey', 'You son of a bitch!']",True
688,2_608.wav,"You are right, by saying nice, I am virtually ...","['Did I go to this school?', ""Hey, there's Mis...",True


In [27]:
def create_mfcc(filepath:str, n_fft:int, hop_length: int, n_mfcc:int = 13):
    """
    Creates MFCC for file at filepath

    :param filepath: Location of file to be used
    :param n_fft: Number of Fast Fourier Transforms
    :param hop_length: Number of Hops within samples
    :param n_mfcc: Number of MFCC's to be outputted
    :return: Array containing mean of all MFCC's
    """
    signal, sample_rate = librosa.load(filepath)
    mfccs = librosa.feature.mfcc(y=signal, sr=sample_rate, n_fft=n_fft, hop_length=hop_length, n_mfcc=n_mfcc)
    mean_mfccs = np.mean(mfccs.T,axis=0)
    return mean_mfccs

In [33]:
def create_padded_text(text:str):
    max_length = 60
    padding_type = 'post'

    tokenizer.fit_on_texts(text)
    train = tokenizer.texts_to_sequences(text)
    padded_train_sequences = pad_sequences(train, maxlen=max_length, padding=padding_type)
    return padded_train_sequences

In [40]:
def load_fasttext_vectors2(fname):
    embeddings_index = {}
    f = codecs.open(fname, encoding='utf-8')
    for line in tqdm(f):
        values = line.rstrip().rsplit(' ')
        ft_word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[ft_word] = coefs
    f.close()
    return embeddings_index

In [34]:
feature_df = pd.DataFrame(columns=["audio_feature","padded_utterance", "is_sarcastic"])

In [35]:
padded_utterances = create_padded_text(dataset['utterance'])

In [36]:
sarcasm_states = dataset['sarcasm']
label_encoder = LabelEncoder()
y_train = to_categorical(label_encoder.fit_transform(sarcasm_states))

In [37]:
hop_length = 512
n_fft = 2048
audio_data_fp = "../mmsd_raw_data/converted_utterances"

for index,row in dataset.iterrows():
    audio_file = row['file_name']
    utterance = row['utterance']
    audio_mfcc = create_mfcc(f"{audio_data_fp}/{audio_file}", n_fft, hop_length)

    feature_df = feature_df.append({
        "audio_feature": audio_mfcc,
        "padded_utterance": padded_utterances[index],
        "is_sarcastic": y_train[index]
    }, ignore_index=True)

In [38]:
feature_df

,audio_feature,padded_utterance,is_sarcastic
0,"[-146.32887, 99.83219, -50.631638, 9.274939, -...","[55, 37, 4, 744, 26, 249, 52, 361, 78, 214, 0,...","[0.0, 1.0]"
1,"[-147.90367, 104.595116, -39.535, 4.108271, -2...","[5, 45, 73, 129, 64, 362, 26, 215, 250, 62, 30...","[0.0, 1.0]"
2,"[-21.120022, 86.11223, -35.381474, 22.640253, ...","[745, 55, 49, 746, 747, 24, 71, 43, 35, 748, 0...","[1.0, 0.0]"
3,"[-23.78374, 70.78856, -34.25742, 23.38274, -16...","[749, 294, 34, 468, 750, 78, 74, 751, 752, 29,...","[1.0, 0.0]"
4,"[1.6350226, 82.73289, -46.96253, 14.140306, -2...","[38, 37, 768, 48, 34, 4, 365, 85, 25, 769, 770...","[0.0, 1.0]"
...,...,...,...
685,"[-273.74615, 118.08891, -53.78213, 41.342915, ...","[2056, 49, 69, 46, 25, 435, 27, 54, 5, 489, 12...","[0.0, 1.0]"
686,"[-100.73689, 113.74322, -51.75491, 9.413455, -...","[36, 60, 65, 150, 4, 2060, 110, 2061, 648, 639...","[1.0, 0.0]"
687,"[-284.09775, 168.39523, -96.155045, 25.642128,...","[34, 30, 40, 184, 25, 2062, 442, 2063, 193, 86...","[0.0, 1.0]"
688,"[-142.58499, 150.03133, -44.35375, 36.23623, -...","[24, 66, 69, 142, 275, 258, 5, 70, 2064, 2065,...","[0.0, 1.0]"


In [44]:
w2v_model = load_fasttext_vectors2("../wiki-news-300d-1M.vec")

999995it [01:31, 10961.78it/s]


{'999994': array([300.], dtype=float32),
 ',': array([ 1.0730e-01,  8.9000e-03,  6.0000e-04,  5.5000e-03, -6.4600e-02,
        -6.0000e-02,  4.5000e-02, -1.3300e-02, -3.5700e-02,  4.3000e-02,
        -3.5600e-02, -3.2000e-03,  7.3000e-03, -1.0000e-04,  2.5800e-02,
        -1.6600e-02,  7.5000e-03,  6.8600e-02,  3.9200e-02,  7.5300e-02,
         1.1500e-02, -8.7000e-03,  4.2100e-02,  2.6500e-02, -6.0100e-02,
         2.4200e-01,  1.9900e-02, -7.3900e-02, -3.1000e-03, -2.6300e-02,
        -6.2000e-03,  1.6800e-02, -3.5700e-02, -2.4900e-02,  1.9000e-02,
        -1.8400e-02, -5.3700e-02,  1.4200e-01,  6.0000e-02,  2.2600e-02,
        -3.8000e-03, -6.7500e-02, -3.6000e-03, -8.0000e-03,  5.7000e-02,
         2.0800e-02,  2.2300e-02, -2.5600e-02, -1.5300e-02,  2.2000e-03,
        -4.8200e-02,  1.3100e-02, -6.0160e-01, -8.8000e-03,  1.0600e-02,
         2.2900e-02,  3.3600e-02,  7.1000e-03,  8.8700e-02,  2.3700e-02,
        -2.9000e-02, -4.0500e-02, -1.2500e-02,  1.4700e-02,  4.7500e-02,
     